# Определение выгодного тарифа для телеком компании

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Необходимо построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»).

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Модель должна быть с максимально большим значением accuracy. Нужно довести долю правильных ответов по крайней мере до 0.75. 
Проверим accuracy на тестовой выборке.

Описание данных
*	сalls — количество звонков,
*	minutes — суммарная длительность звонков в минутах,
*	messages — количество sms-сообщений,
*	mb_used — израсходованный интернет-трафик в Мб,
*	is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

# План проекта
* [1. Откроем и изучим файл](#1)
* [2. Разобьем данные на выборки](#2)
* [3. Исследуем модели](#3)
* [4. Проверим модель на тестовой выборке](#4)
* [5. Проверим модели на адекватность](#5)
* [6. Проверим модели на адекватность](#6)
* [7. Чек-лист готовности проекта](#7)

<a name="1"></a>
## Откроем и изучим файл

In [3]:
# загрузим библиотеки
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [4]:
# Чтение данных из файла
df = pd.read_csv('/datasets/users_behavior.csv')

In [5]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [6]:
# данные предобработаны, но на всякий случай проверим на пропуски
pd.DataFrame(round((df.isna().mean()*100),2)).style.background_gradient('coolwarm')

,0
calls,0
minutes,0
messages,0
mb_used,0
is_ultra,0


In [7]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Предобработку сделана ранее. Данные имеют нужный формат, пропусков нет, можно выполнять основную задачу.

<a name="2"></a>
## Разобьем данные на выборки

Давайте разделим датафрейм на 3 части: обучающую, валидационную и тестовую выбороки в пропорциях 3:1:1.
Делим вначале df на df_train и df_valid_test, потом df_valid_test на df_valid и df_test

In [8]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)

In [9]:
# разделим выборки на признаки и целевой признак
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_valid.drop(['is_ultra'], axis=1)
target_test = df_valid['is_ultra']

In [10]:
# посмотрим на корректность столбцов после разбиений и разделений
features_train.head()

,calls,minutes,messages,mb_used
3027,60.0,431.56,26.0,14751.26
434,33.0,265.17,59.0,17398.02
1226,52.0,341.83,68.0,15462.38
1054,42.0,226.18,21.0,13243.48
1842,30.0,198.42,0.0,8189.53


In [12]:
# выведем размеры выборок для контроля
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


### Данные разбили на выборки. Разделили выборки на признаки и целевые признаки.

<a name="3"></a>
## Исследуем модели

Наша задача относится к задаче классификации. Точнее бинарной классификации. 
Давайте рассмотрим 3 модели:

Решающее дерево DecisionTreeClassifier.

Случайный лес RandomForestClassifier.

Логистическая регрессия LogisticRegression.

### Решающее дерево DecisionTreeClassifier.

In [149]:
# создадим модель и обучим
model_decision_tree = DecisionTreeClassifier(random_state=12345)
model_decision_tree.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

In [150]:
# получим предсказания модели на валидационной выборке
tree_valid_predictions = model_decision_tree.predict(features_valid)

In [151]:
# Получим оценку точности предсказания accuracy
tree_accuracy = accuracy_score(target_valid, tree_valid_predictions)
tree_accuracy

0.713841368584759

In [153]:
# изменим гиперпараметры и оценим, как они влияют на accuracy.
# возьмем максимальную глубину до 10
for depth in range(1,11):
    model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_decision_tree.fit(features_train, target_train)
    tree_valid_predictions = model_decision_tree.predict(features_valid)
    print('depth', depth, ',', 'accuracy', ':', accuracy_score(target_valid, tree_valid_predictions))

depth 1 , accuracy : 0.7542768273716952
depth 2 , accuracy : 0.7822706065318819
depth 3 , accuracy : 0.7853810264385692
depth 4 , accuracy : 0.7791601866251944
depth 5 , accuracy : 0.7791601866251944
depth 6 , accuracy : 0.7838258164852255
depth 7 , accuracy : 0.7822706065318819
depth 8 , accuracy : 0.7791601866251944
depth 9 , accuracy : 0.7822706065318819
depth 10 , accuracy : 0.7744945567651633


In [154]:
# создадим итоговую, оптимальную модель и обучим
model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=3)
model_decision_tree.fit(features_train, target_train)
# получим предсказания модели на валидационной выборке
tree_valid_predictions = model_decision_tree.predict(features_valid)
# Получим оценку точности предсказания accuracy
tree_accuracy = accuracy_score(target_valid, tree_valid_predictions)
tree_accuracy

0.7853810264385692

#### При увеличении глубины, качество модели не улучшается. Модель "Решающее дерево" склонна к переобучению. Оптимальное значение: depth = 3 , accuracy = 0.785

### Случайный лес - RandomForestClassifier

In [155]:
# создадим модель с количеством деревьев n_estimators=5 и обучим ее
model_random_forest = RandomForestClassifier(random_state=67895, n_estimators=5)
model_random_forest.fit(features_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=67895,
                       verbose=0, warm_start=False)

In [156]:
# получим предсказания модели на валидационной выборке
forest_valid_predictions = model_random_forest.predict(features_valid)

In [157]:
# Получим оценку точности предсказания accuracy
forest_accuracy = accuracy_score(target_valid, forest_valid_predictions)
forest_accuracy

0.7480559875583204

In [158]:
# меняя гиперпараметры, посмотрим, как они повлияют на качество модели. Давайте изменим критерий Джини на энтропию
model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=5, criterion='entropy')
model_random_forest.fit(features_train, target_train)
forest_valid_predictions = model_random_forest.predict(features_valid)
forest_accuracy = accuracy_score(target_valid, forest_valid_predictions)
forest_accuracy

0.7620528771384136

качество модели улучшилось чуть более, чем на 1%

In [159]:
# изменим max_depth - максимальную глубину дерева. Переберем в цикле значения от 1 до 20
for depth in range(1,21):
    model_random_forest = RandomForestClassifier(random_state=67895, n_estimators=5, max_depth=depth, criterion='entropy')
    model_random_forest.fit(features_train, target_train)
    forest_valid_predictions = model_random_forest.predict(features_valid)
    print('depth', depth, ',', 'accuracy', accuracy_score(target_valid, forest_valid_predictions))

depth 1 , accuracy 0.7589424572317263
depth 2 , accuracy 0.7900466562986003
depth 3 , accuracy 0.7869362363919129
depth 4 , accuracy 0.7916018662519441
depth 5 , accuracy 0.7916018662519441
depth 6 , accuracy 0.7962674961119751
depth 7 , accuracy 0.7916018662519441
depth 8 , accuracy 0.7916018662519441
depth 9 , accuracy 0.7791601866251944
depth 10 , accuracy 0.7853810264385692
depth 11 , accuracy 0.807153965785381
depth 12 , accuracy 0.7838258164852255
depth 13 , accuracy 0.7838258164852255
depth 14 , accuracy 0.7698289269051322
depth 15 , accuracy 0.7682737169517885
depth 16 , accuracy 0.7729393468118196
depth 17 , accuracy 0.7589424572317263
depth 18 , accuracy 0.7682737169517885
depth 19 , accuracy 0.7480559875583204
depth 20 , accuracy 0.7822706065318819


При увеличении глубины, качество модели значимо не улучшается. Оптимальное значение depth = 11, accuracy = 0.807

In [160]:
# изменим n_estimators - количество деревьев. Переберем в цикле от 5 до 50, с шагом 5
for estimators in range(5,51,5):
    model_random_forest = RandomForestClassifier(random_state=67895, n_estimators=estimators, max_depth=11, criterion='entropy')
    model_random_forest.fit(features_train, target_train)
    forest_valid_predictions = model_random_forest.predict(features_valid)
    print('estimators', estimators,',', 'accuracy', accuracy_score(target_valid, forest_valid_predictions))


estimators 5 , accuracy 0.807153965785381
estimators 10 , accuracy 0.7978227060653188
estimators 15 , accuracy 0.7993779160186625
estimators 20 , accuracy 0.7978227060653188
estimators 25 , accuracy 0.7993779160186625
estimators 30 , accuracy 0.7993779160186625
estimators 35 , accuracy 0.7993779160186625
estimators 40 , accuracy 0.7993779160186625
estimators 45 , accuracy 0.7993779160186625
estimators 50 , accuracy 0.8040435458786936


Оптимально n_estimators = 5 , accuracy= 0.807. Далее, при увеличении количества деревьев, качество хуже, а скорость уменьшается.

In [161]:
# запустим и проверим модель с итоговыми лучшими параметрами
model_random_forest = RandomForestClassifier(random_state=67895, n_estimators=5, max_depth=11, criterion='entropy')
model_random_forest.fit(features_train, target_train)
forest_valid_predictions = model_random_forest.predict(features_valid)
forest_accuracy = accuracy_score(target_valid, forest_valid_predictions)
forest_accuracy

0.807153965785381

#### При помощи подбора гиперпараметров настроили качество модели "Случайный лес - RandomForestClassifier" с 74% до 80%.

### Логистическая регрессия - LogisticRegression

In [162]:
# создадим модель
logistic_model = LogisticRegression(random_state=67895)
# обучим
logistic_model.fit(features_train, target_train)
# получим предсказания
logistic_valid_predictions = logistic_model.predict(features_valid)
# оценим качество модели
logistic_accuracy = accuracy_score(target_valid, logistic_valid_predictions)
logistic_accuracy

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7589424572317263

In [163]:
# изменим гиперпараметры solver='liblinear', warm_start=True
logistic_model = LogisticRegression(random_state=67895, solver='liblinear', warm_start=True)
logistic_model.fit(features_train, target_train)
logistic_valid_predictions = logistic_model.predict(features_valid)
logistic_accuracy = accuracy_score(target_valid, logistic_valid_predictions)
logistic_accuracy

0.7589424572317263

Изменение гиперпараметров не повлияло на качество модели.

### Мы создали, обучили, получили предсказания и оценили качество 3-х моделей.
* Решающее дерево DecisionTreeClassifier. Наилучшие результаты при depth = 3 , accuracy = 0.785
* Случайный лес RandomForestClassifier. При помощи подбора гиперпараметров настроили качество модели до accuracy = 0.807
* Логистическая регрессия - LogisticRegression. Настройка гиперпараметров не помогла, accuracy = 0.759.
* В дальнейшем будем использовать модель Случайный лес RandomForestClassifier, как самую качественную.

<a name="4"></a>
## Проверим модель на тестовой выборке 

In [121]:
# Разделим тестовый датафрейм на features и target - целевой признак
features_test = df_test.drop(['is_ultra'], axis=1)  
target_test = df_test['is_ultra']

In [164]:
# получим предсказания
predictions = model_random_forest.predict(features_test)
# оценим качество модели
accuracy = accuracy_score(target_test, predictions)
accuracy

0.807153965785381

In [165]:
# можно еще так посчитать
model_random_forest.score(features_test, target_test)

0.807153965785381

In [166]:
# для сравнения давайте проверим точность на модели "Решающее дерево DecisionTreeClassifier"
model_decision_tree.score(features_test, target_test)

0.7853810264385692

### Модель случайный лес ожидаемо отработала с лучшим качеством, чем решающее дерево. Точность на тестовой выборке близка к исходной. Это значит, что модель работает адекватно. Она не переучена.

<a name="5"></a>
## Проверим модели на адекватность

В библиотеке scikit-learn есть DummyClassifier() для задач классификации и DummyRegressor() для работы с регрессией. В нашем случае задача классификации, берем DummyRegressor() 

In [167]:
# создадим, обучим и оценим качество модели для наших тестовых данных
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_test, target_test)
dummy_clf.score(features_test, target_test)

0.7060653188180405

Отличный результат! Но, наша модель "Случайный лес", лучше. Ее точность на тестовой выборке 0.796. Результат проверки - наша модель адекватна!

<a name="6"></a>
## Общий вывод

* Задача проекта: создать модель с точностью предсказаний выше 0.75. Задача выполнена. Модель создана.
* Итоговая модель "Случайный лес" проверена на тестовой выборке и проверена на адекватность. Все проверки прошли успешно.
* Проанализировав поведение клиентов, наша модель с точность 80% может предложить пользователям новый тариф: «Смарт» или «Ультра».